In [4]:
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time

import sys

firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities['marionette'] = True
firefox_capabilities['handleAlerts'] = True
firefox_capabilities['acceptSslCerts'] = True
firefox_capabilities['acceptInsecureCerts'] = True
geckoPath = 'driver/geckodriver.exe'
#capabilities=firefox_capabilities,
firefox = webdriver.Firefox(executable_path=geckoPath)
#driver = webdriver.PhantomJS(executable_path='driver/phantomjs.exe')
driver = webdriver.Firefox( executable_path=geckoPath)
try:
    firefox.get('https://www.crunchbase.com/organization/apple')
except:
    pass
driver.get('https://techcrunch.com/')
firefox.set_page_load_timeout(2)
driver.set_page_load_timeout(30)

def existence_in_crunchbase(name):
    base_url = 'www.crunchbase.com'
    url = 'https://techcrunch.com/search/'+name
    url_crunchbase = None
    try:
        driver.get(url)
    except KeyboardInterrupt:
        sys.exit()
    except:
        print('timeout of phantomjs')
        pass
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    blocks = soup.select("h2.post-title a")
    if len(blocks)>0:
        url_crunchbase = blocks[0]["href"]  
        name  = blocks[0].getText()
 
        if base_url not in url_crunchbase:
            url_crunchbase = None
    
    return url_crunchbase
            

def extract_company_techcrunch(name,url):
    try:
        firefox.get(url)
    except:
        pass
    founded = None
    employees = None
    company_name = ''
    blocks_dt = []
    blocks_dd = []
    try:
        soup = BeautifulSoup(firefox.page_source, "html.parser")
        blocks_dt = soup.select("div.details dt")
        blocks_dd = soup.select("div.details dd")
        company_name_tag = soup.select_one("#profile_header_heading")
        if company_name_tag == None:
            company_name = ''
        else:
            company_name = company_name_tag.getText()
    except:
        pass

    i = 0
    while company_name.lower() != name.lower() or len(blocks_dt)==0 :
        time.sleep(0.3)
        try:
            soup = BeautifulSoup(firefox.page_source, "html.parser")
            company_name_tag = soup.select_one("#profile_header_heading")
            if company_name_tag:
                company_name = company_name_tag.getText()
            else:
                company_name = ''
            blocks_dt = soup.select("div.details dt")
            blocks_dd = soup.select("div.details dd")
        except KeyboardInterrupt:
            sys.exit()
        except:
            pass
        i=i+1
        if i >= 100:
            print('fail to crawl ', name , ' in crunchbase')
            break

    
    if company_name.lower() == name.lower():
        for index,block in enumerate(blocks_dt):
            if 'Founded' in  block.getText() : 
                founded = blocks_dd[index].getText()
            if 'Employees' in block.getText():
                employees = blocks_dd[index].getText().split('|')[0]
    print( name , founded , employees)
    return founded,employees



In [6]:
import time
import json
import os

class CompaniesManager:
    def __init__(self):
        self.companies = []
        self.articles = []
        self.companies_name = []
        self.load_articles()
        self.load_companies()
        
    def load_articles(self):
        if os.path.isfile('data/raw_articles.json'):
            f = open('data/raw_articles.json')
            self.articles = json.load(f)
            f.close()
            
    def load_companies(self):
        if os.path.isfile('data/raw_companies.json'):
            f = open('data/raw_companies.json')
            self.companies = json.load(f)
            f.close()      
            for company in self.companies:
                name = company["name"]
                self.companies_name.append(name)
        
            
    def extract_companies(self):
        
        for index,article in enumerate(self.articles):
            for company_name in article["companies"]: 
                article_extraInfos = article["extra_infos"]
                for info in article_extraInfos:
                    if info["text"]== company_name:
                        relevance = info["relevance"]
                        count_in_article = info["count"]
                            
                if company_name in self.companies_name :
                    for company in self.companies:
                        if company_name == company["name"] and article["id"] not in company["articles"]: 
                            company["count"] = company["count"]+1
                            company["sentiment"]= company["sentiment"]+article["sentiment"]
                            company["articles"].append(article["id"])
                            extra_infos = {
                                 "id":article["id"] ,
                                 "count_in_article":count_in_article,
                                 "revelance": relevance       
                            }
                            company["extra_infos"].append(extra_infos)
                            
                else:
                    company = {
                        "name": company_name,
                        "dateFound": int(str(time.time()).split('.')[0]),
                        "count":1,
                        "sentiment": article["sentiment"],
                        "articles": [article["id"]],
                        "extra_infos":
                        [
                            {
                             "id": article["id"],
                             "count_in_article":count_in_article,
                             "revelance": relevance
                            }  
                        ]    
                    }
                    self.companies.append(company)
                    self.companies_name.append(company_name)
            
        self.save_to_disk()
        
    def extend_crunch(self):
        
        i = 0
        for index,company in enumerate(self.companies):
            name = company["name"]
            search_label = "0"
            if "search_label" in company:
                search_label = company["search_label"]
                
            if search_label=="0" and index >2900:
                url_crunchbase = existence_in_crunchbase(name)
                
                if url_crunchbase != None:
                    print(name, ' in crunchbase')
                    i=i+1
                    founded,employees = extract_company_techcrunch(name,url_crunchbase)
                    company["search_label"] = url_crunchbase
                    if founded is not None:
                        company["foundationDate"] = founded
                    if employees is not None:
                        company["number_of_employees"] = employees
                    
                else:
                    company["search_label"] = str(0)
                    print(name, 'not in crunchbase')
            else:
                print("skip: ", name)
            
            if (index+1)%100 == 0:
                self.save_to_disk()
        self.save_to_disk()            
                                
    def save_to_disk(self):
        with open('data/raw_companies.json', 'w') as company_file:
            json.dump(self.companies, company_file,indent = 2)
            company_file.close()
                
                
                
            
    

In [7]:
CM = CompaniesManager()
#CM.extract_companies()
CM.extend_crunch()


skip:  SENSORO
skip:  Microsoft Corp
skip:  Sumitomo
skip:  Manchester City Verve
skip:  SalesWings
skip:  SA tech
skip:  Avito
skip:  Tshimologong Innovation Hub
skip:  Flickr
skip:  Niantic Labs
skip:  Oblong
skip:  Osterhout Design Group
skip:  Niantic
skip:  Oblong Industries
skip:  Mezzanine
skip:  The Pokemon Company
skip:  GamesBeat
skip:  BNJ Marketing
skip:  DormaKaba
skip:  Berkshire companies
skip:  Bezos
skip:  Private Equity
skip:  Utrip
skip:  Greylock
skip:  Elsight
skip:  LiveU
skip:  WAN
skip:  Vodaphone
skip:  Optus
skip:  GoRemote
skip:  Agri Alliance
skip:  Bitbond
skip:  Obotritia
skip:  FiveAI
skip:  GuestReady
skip:  Carspring
skip:  Moneybox
skip:  Drivetribe
skip:  Realla
skip:  Habito
skip:  Sling & Stone
skip:  Crowdcube
skip:  Cali Rice
skip:  Nominet Trust
skip:  3D Robotics
skip:  AgFunder
skip:  Airware
skip:  Arbe Robotics
skip:  Zipline International
skip:  Redbird
skip:  Bloomberg Businessweek
skip:  Yuneec Electric Aviation
skip:  CyPhy Works
skip:  E

skip:  Paylocity
skip:  MetricStream
skip:  Kaiser Permanente
skip:  McKinsey
skip:  Butterfly Network
skip:  General Motors Partners
skip:  Blue Apron
skip:  Goodnow
skip:  Virtual Logistics
skip:  KidsToyStop.com
skip:  CutCableToday.com
skip:  ITFT
skip:  North Carolina-based
skip:  Memphis Meats
skip:  Mosa Meats
skip:  Recall Masters
skip:  Criterion Networks
skip:  Washington Mutual
skip:  Nissan
skip:  EchoStar
skip:  Entrepreneur Network
skip:  Roku
skip:  Wizpak
skip:  Voice Labs
skip:  Dow Jones
skip:  Seaters Inc.
skip:  Fiverr
skip:  True & Co.
skip:  Polyvore
skip:  Ramamurthy
skip:  Lumoid
skip:  theBoardlist
skip:  Tumml
skip:  Echo
skip:  Bespoken Tools
skip:  Conversible
skip:  Rocksauce Studios
skip:  TensorFlow
skip:  dLocal
skip:  Land Rover
skip:  Chipotle
skip:  J Crew
skip:  Chatbots Work Botsify
skip:  ChattyPeople
skip:  ZenDesk
skip:  Tucker Toys
skip:  Sensible Objects
skip:  InventRight, LLC.
skip:  CSID
skip:  Tesla factory
skip:  Foster + Partners
skip:  V

skip:  OneEnergy
skip:  OneEnergy Renewables
skip:  Portland General Electric
skip:  Climate Solutions
skip:  Apptentive
skip:  Sift Science
skip:  Tableau
skip:  LaRubbio
skip:  Closer Mobile
skip:  Urbanhail
skip:  Nohla
skip:  Sensu
skip:  Heavy Water
skip:  Sonian
skip:  Concur SAP
skip:  MetaBrite
skip:  Kraft
skip:  ShelfEngine
skip:  Jobaline
skip:  iConclude
skip:  Market Leader
skip:  Amplify Partners
skip:  Opsware
skip:  Xnor.ai
skip:  AI2
skip:  Convoy
skip:  Bezos Expeditions
skip:  Concur
skip:  StratusCore
skip:  Chaos Group
skip:  Jeeva Wireless
skip:  Jeeva
skip:  Vantron Technology
skip:  InKind
skip:  Standing Rock
skip:  Zealyst
skip:  Kataoka
skip:  ConveneVR
skip:  Welkhoff
skip:  Pavia Systems
skip:  Eileses
skip:  Samepage Health
skip:  Health123
skip:  Mayo Clinic
skip:  MOHAI
skip:  PetHub
skip:  Arcview Market Research
skip:  Privateer Holdings
skip:  Leafly
skip:  Poseidon Asset Management
skip:  NPD
skip:  FlexMinder
skip:  Jellyvision
skip:  GMI
skip:  Tol

skip:  Learning Machine
skip:  Blockchain Global
skip:  IBM Australia
skip:  Hyperledger Project
skip:  CheckInSafe
skip:  Arludo
skip:  ReadyGoFund
skip:  ReadyFundGo
skip:  Stake
skip:  CommSec
skip:  Sanlam Private Wealth
skip:  Optiver
skip:  ASX
skip:  Macquarie
skip:  ETF Securities
skip:  Bluescope
skip:  Springboard Enterprises Australia
skip:  Specix
skip:  Regional Innovation Fund
skip:  fintech
skip:  Faethm
skip:  HR Tech
skip:  Opentute
skip:  iComplii
skip:  Spacely
skip:  Servcorp
skip:  iProperty
skip:  Regus
skip:  Airsorted
skip:  Audacious Digital
skip:  Colgate Palmolive
skip:  Telstra Labs
skip:  Yik Yak
skip:  BigCommerce
skip:  WME Group
skip:  Outware Systems
skip:  Netregistry
skip:  Appscore
skip:  Ideapod
skip:  zipMoney
skip:  Afterpay
skip:  DeakinCo
skip:  BuddingTech
skip:  Kontented
skip:  ROI Kontented
skip:  APN News
skip:  Coca Cola
skip:  Airwallex
skip:  Sequoia Partner
skip:  Gobi Partners
skip:  Bugdojo
skip:  Bugwolf
skip:  Bitbucket
skip:  Airta

skip:  TicketMaster
skip:  NuCurrent
skip:  CardRunners Gaming
skip:  Development Loans
skip:  WedgeRock
skip:  bluShift
skip:  Intrepid Travel
skip:  Bookmundi
skip:  Sidekix
skip:  Elev8 Consulting Group
skip:  Calix Networks
skip:  FetchitGO
skip:  DIY SD
skip:  Logitech
skip:  Obi phones
skip:  WebPT
skip:  SoCal
skip:  Universal Mind
skip:  Surge Hydro
skip:  Thrivant Health
skip:  Biddeford Savings
skip:  Callbox Storage
skip:  Conversable
skip:  4DHealthware
skip:  Rise Science
skip:  Intraboom
skip:  APAC
skip:  Somavac Medical Solutions
skip:  Somavac
skip:  HABERDAShe
skip:  Delta
skip:  Blue Mail Media
skip:  Vimeo
skip:  SquareFoot
skip:  ShearShare
skip:  Rayton Solar
skip:  Patexia
skip:  Buchwald & Associates
skip:  SPLT
skip:  WiseBanyan
skip:  SXSW
skip:  LaunchKC
skip:  Sweek
skip:  Ravensburger
skip:  Mybestseller
skip:  Fabric Premium
skip:  Fabric
skip:  Vantis Life
skip:  Cocoa Belle Chocolates
skip:  Cocoa Belle
skip:  RaftUp
skip:  Ooma
skip:  Company
skip:  Atl

UBM not in crunchbase
IndieBio not in crunchbase
TechCode not in crunchbase
Canon Startup Company not in crunchbase
XTC not in crunchbase
Hemostatic not in crunchbase
Vantage Robotics  in crunchbase
Vantage Robotics 2013 1 - 10 
Omnicom not in crunchbase
Nexeon Medsystems  in crunchbase
Nexeon Medsystems 2005 1 - 10 
HiFly Photography not in crunchbase
Cosmunity  in crunchbase
Cosmunity 2016 1 - 10 
Immerss  in crunchbase
Immerss July 1, 2013 1 - 10 
Dallas-based Loopback Analytics not in crunchbase
Humach  in crunchbase
Humach Unknown 251 - 500 
MarryMyCity not in crunchbase
NewBo City Market not in crunchbase
Octane AI  in crunchbase
Octane AI May, 2016 4 in Crunchbase
Blippar  in crunchbase
Blippar August 17, 2011 251 - 500 
Kuli Kuli Foods not in crunchbase
LumaStream  in crunchbase
LumaStream 2009 11 - 50 
Urban Industrial not in crunchbase
Zoom Interiors  in crunchbase
Zoom Interiors January 6, 2013 11 - 50 
SB Projects  in crunchbase
SB Projects Unknown 1 in Crunchbase
FF Angel 

Petcube May 26, 2012 11 - 50 
DigitalOcean  in crunchbase
DigitalOcean June 24, 2011 251 - 500 
Rackspace  in crunchbase
Rackspace 1998 59 in Crunchbase
Fundera  in crunchbase
Fundera October 1, 2013 51 - 100 
Watson Marketing EMEA not in crunchbase
Blink  in crunchbase
Blink May 10, 2015 1 - 10 
Opcity  in crunchbase
Opcity December, 2015 101 - 250 
Berkshire Hathaway Home Services not in crunchbase
Web.com  in crunchbase
Web.com 1997 1k - 5k 
Georgian Partners  in crunchbase
Georgian Partners 2008 None
ReMax  in crunchbase
ReMax 1973 10k + 
Better Homes  in crunchbase
Better Homes July 9, 1986 5k - 10k 
IdealSeat  in crunchbase
IdealSeat 2012 1 - 10 
NASCAR  in crunchbase
NASCAR December 14, 1947 5k - 10k 
CleanTurn Enterprises not in crunchbase
Dispatch Media Group not in crunchbase
CleanTurn Demolition Services not in crunchbase
CleanTurn not in crunchbase
Auditgraph not in crunchbase
SHAN Cleaning Services not in crunchbase
VEGA Coffee  in crunchbase
VEGA Coffee January 19, 2014 1

Suade August 31, 2014 1 - 10 
Kompli not in crunchbase
Aosphere not in crunchbase
Wild Flowers not in crunchbase
Splittable not in crunchbase
ManoMano  in crunchbase
ManoMano June 19, 2013 101 - 250 
Captify  in crunchbase
Captify July 4, 2011 51 - 100 
Reach Robotics  in crunchbase
Reach Robotics May 20, 2013 1 - 10 
Cambridge Medical Robotics  in crunchbase
Cambridge Medical Robotics 2014 11 - 50 
startup Automata not in crunchbase
Moley Robotics  in crunchbase
Moley Robotics 2015 5 in Crunchbase
Consequential Robotics  in crunchbase
Consequential Robotics Unknown 4 in Crunchbase
MiRo  in crunchbase
MiRo 2011 None found in Crunchbase
RAAS not in crunchbase
Open Bionics  in crunchbase
Open Bionics 2014 1 - 10 
Innovate UK not in crunchbase
Oxford Technology not in crunchbase
Primo Toys  in crunchbase
Primo Toys June 1, 2013 11 - 50 
Emotech not in crunchbase
Constant Contact  in crunchbase
Constant Contact 1998 5k - 10k 
SendBlaster  in crunchbase
SendBlaster Unknown 11 - 50 
Sendinbl

SystemExit: 

d:\code\python3.5\lib\site-packages\IPython\core\interactiveshell.py:2855: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
CM.save_to_disk()